In [1]:
from typing import List


def validate_extended_matrix(matrix: List[List[float]], n: int) -> None:
    extended_length = n + 1
    for i in range(n):
        if len(matrix[i]) != extended_length:
            raise Exception(f'Incorrect matrix was passed. Expected length {extended_length} on row {i}.')


def copy_matrix(matrix: List[List[float]]) -> List[List[float]]:
    return [[col for col in row] for row in matrix]


def solve_gaus(matrix: List[List[float]]) -> List[float]:
    n = len(matrix)
    validate_extended_matrix(matrix, n)
    operative_matrix = copy_matrix(matrix)

    for i in range(n):
        for j in range(i + 1, n):
            v = operative_matrix[j][i] / operative_matrix[i][i]
            for k in range(i, n + 1):
                operative_matrix[j][k] -= v * operative_matrix[i][k]

    ans = [0 for _ in range(n)]
    for i in reversed(range(n)):
        v = operative_matrix[i][n]
        for j in range(i + 1, n):
            v -= operative_matrix[i][j] * ans[j]

        ans[i] = v / operative_matrix[i][i]

    return ans


def solve_gaus_jordan(matrix: List[List[float]]) -> List[float]:
    n = len(matrix)
    validate_extended_matrix(matrix, n)
    operative_matrix = copy_matrix(matrix)

    for i in range(n):
        for j in filter(lambda arg1: i != arg1, range(n)):
            v = operative_matrix[j][i] / operative_matrix[i][i]
            for k in range(i, n + 1):
                operative_matrix[j][k] -= v * operative_matrix[i][k]

    ans = [row[n] / row[i] for i, row in enumerate(operative_matrix)]
    return ans


ans_gaus_1 = solve_gaus([[9, -6, -2, 10], [-2, 8, -3, 0], [-1, -4, 6, 0]])
ans_gaus_jordan_1 = solve_gaus_jordan([[9, -6, -2, 10], [-2, 8, -3, 0], [-1, -4, 6, 0]])
ans_gaus_2 = solve_gaus([[-1, -1, 6, 2], [-1, 6, -1, 34], [6, -1, -1, 12]])
ans_gaus_jordan_2 = solve_gaus_jordan([[-1, -1, 6, 2], [-1, 6, -1, 34], [6, -1, -1, 12]])
ans_gaus_3 = solve_gaus([[1, 4, -3, 4], [4, 20, 14, 20], [3, 14, 14, 14]])
ans_gaus_jordan_3 = solve_gaus_jordan([[1, 4, -3, 4], [4, 20, 14, 20], [3, 14, 14, 14]])

print(ans_gaus_1)
print(ans_gaus_jordan_1)
print(ans_gaus_2)
print(ans_gaus_jordan_2)
print(ans_gaus_3)
print(ans_gaus_jordan_3)


[1.7821782178217822, 0.7425742574257427, 0.7920792079207922]
[1.7821782178217822, 0.7425742574257427, 0.7920792079207922]
[3.428571428571429, 6.571428571428571, 2.0]
[3.428571428571429, 6.571428571428571, 2.0]
[0.0, 1.0, 0.0]
[0.0, 1.0, 0.0]
